In [139]:
import json
import numpy as np
import gurobipy
import utils
from gurobipy import Model, GRB

In [141]:
m, data = utils.create_model('instances/toy_instance.json')

In [145]:
solutions = []
m.params.outputflag = 0
m.NumScenarios = (data.Nj+1)*(data.Nm*data.Np + 1)

c1 = m.addConstr(data.f2 == 0)
c2 = m.addConstr(data.f3 == 0)

for i in range(data.Nj+1):
    for j in range(data.Nm*data.Np+1):
        m.params.ScenarioNumber = i*(data.Nm*data.Np+1) + j
        m.ScenNName = 'i = {}, j = {}'.format(i, j)
        c1.ScenNRhs = j
        c2.ScenNRhs = i

print("Starting optimzation...")

m.setObjective(data.f1)
m.reset()
m.optimize()

print("done.")

for s in range(m.NumScenarios):
    m.params.ScenarioNumber = s
    solutions.append([m.ScenNObjVal, data.Af.ScenNX.sum(), data.Dm.ScenNX])

m.remove(c1)
m.remove(c2)



solutions = np.array(solutions)
solutions = solutions.round().astype(int)

Starting optimzation...
done.


In [166]:
filtered_solutions = []

def dominate(s1, s2):
    temp = s1 - s2
    if max(temp) > 0 or min(temp) == 0:
        return False
    
    return True

for i in range(solutions.shape[0]):
    non_dominated = True
    for j in range(solutions.shape[0]):
        if i == j:
            continue
        if dominate(solutions[j], solutions[i]):
            non_dominated = False        
            break
    if non_dominated:
        filtered_solutions.append(solutions[i])
        
filtered_solutions = np.array(filtered_solutions)
    

In [176]:
import plotly.express as px
import pandas as pd

data_plot = pd.DataFrame(filtered_solutions)
fig = px.scatter_3d(x=data_plot[0], y=data_plot[1], z=data_plot[2])
fig.show()